In [28]:
#利用LSTM做NER，参考了pytorch官网文档

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from pandas import DataFrame
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [29]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = []
testing_data = []

filepath = "train.csv"
train = pd.read_csv(filepath)
test = pd.read_csv("test.csv")


for line in range(0, len(train["text"])):   
    temp1, temp2 = [one for one in train["text"][line]], train["BIO_anno"][line].split()   #将字符个数和bio个数不等的去掉
    if len(temp1) == len(temp2):
        training_data.append(([one for one in train["text"][line]], train["BIO_anno"][line].split()))

for line in range(0, len(test["text"])):
    testing_data.append([one for one in test["text"][line]])

word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
#print(word_to_ix)

for sent in testing_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"B-BANK": 0, "I-BANK": 1, "B-PRODUCT": 2, "I-PRODUCT": 3, "O": 4, "B-COMMENTS_N": 5, "I-COMMENTS_N": 6, "B-COMMENTS_ADJ": 7, "I-COMMENTS_ADJ": 8}  # Assign each tag with a unique index

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 32  #要增大embedding dim，否则超出index报错
HIDDEN_DIM = 32

In [30]:
for sentens, tags in training_data:    #判断是否由个数不相等的
    if len(sentens) != len(tags):
        print('error!')

In [31]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [32]:
print(len(word_to_ix))

2054


In [33]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix)).to(device) #将模型转到gpu上运行
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()

for epoch in range(100):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix).to(device)  #将输入数据弄到gpu上
        targets = prepare_sequence(tags, tag_to_ix).to(device)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()



In [ ]:
# See what the scores are after training
model = model.to('cpu')  #再将模型转为cpu运行
tag_scores_all = []
with torch.no_grad():
    for i in range(0,len(training_data)):
        inputs = prepare_sequence(training_data[i][0], word_to_ix)
        tag_scores = model(inputs)
        tag_scores_all.append(tag_scores)
        #将scores保存在tag_scores_all中

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    #print(tag_scores_all)
    
ans_all = []  #存储训练集预测的scores矩阵
#训练集预测
for num in range(0,len(training_data)):
    max_indexs = []
    ans = []
    for l in range(tag_scores_all[num].shape[0]):  
        #获取scores数组中的最大index，即可以转化称为命名实体
        list1 = list(tag_scores_all[num][l])
        max_val = max(tag_scores_all[num][l])
        max_index = list1.index(max_val)
        max_indexs.append(max_index)  #一句话的对应的最大索引
    for index in max_indexs:
        ans.append(list(tag_to_ix.keys())[index])
    ans_all.append(ans)#所有text对应的命名实体

print(ans_all[0])


['B-BANK', 'I-BANK', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-COMMENTS_N', 'I-COMMENTS_N', 'O', 'O', 'O', 'O', 'O', 'B-COMMENTS_ADJ', 'I-COMMENTS_ADJ', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-COMMENTS_N', 'I-COMMENTS_N', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-COMMENTS_N', 'O', 'O', 'O', 'O', 'B-PRODUCT', 'I-PRODUCT', 'O', 'O', 'O', 'O', 'B-COMMENTS_ADJ', 'I-COMMENTS_ADJ', 'O', 'O', 'O', 'O', 'O', 'O', 'B-COMMENTS_N', 'O', 'B-COMMENTS_N', 'O', 'O', 'O']


In [ ]:
tag_scores_test = []#存储测试集预测的scores矩阵
with torch.no_grad():
    for i in range(0,len(test["text"])):
        inputs = prepare_sequence(testing_data[i], word_to_ix)
        tag_scores = model(inputs)
        tag_scores_test.append(tag_scores)


ans_test = []
for num in range(0,len(test["text"])):
    max_indexs = []
    ans = []
    for l in range(tag_scores_test[num].shape[0]):  
        #获取scores数组中的最大index，即可以转化称为命名实体
        list1 = list(tag_scores_test[num][l])
        max_val = max(tag_scores_test[num][l])
        max_index = list1.index(max_val)
        max_indexs.append(max_index)  #一句话的对应的最大索引
    for index in max_indexs:
        ans.append(list(tag_to_ix.keys())[index])
    ans_test.append(ans)#所有text对应的命名实体
print(ans_test[0])




['O', 'I-COMMENTS_ADJ', 'O', 'O', 'O', 'O', 'B-PRODUCT', 'I-PRODUCT', 'O', 'B-PRODUCT', 'I-PRODUCT', 'I-PRODUCT', 'O', 'B-PRODUCT', 'O', 'B-COMMENTS_N', 'I-COMMENTS_N', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
BIO_anno = str(ans_test[0])
BIO_anno = BIO_anno.replace(',','')
BIO_anno = BIO_anno.replace('\'','')
BIO_anno = BIO_anno.strip('[')
BIO_anno = BIO_anno.strip(']')
print(BIO_anno)

O I-COMMENTS_ADJ O O O O B-PRODUCT I-PRODUCT O B-PRODUCT I-PRODUCT I-PRODUCT O B-PRODUCT O B-COMMENTS_N I-COMMENTS_N O O O O O O O


In [ ]:
#用ans_test改变之前的标注栏目
len(ans_test)
test_update = pd.read_csv('output.csv')
file = open('output1.csv', 'w', encoding='utf8')

out = 'id' + ',' + 'BIO_anno' + ',' + 'class'+'\n'
file.write(str(out))

for line in range(0, len(test_update['BIO_anno'])):
    BIO_anno = str(ans_test[line])
    BIO_anno = BIO_anno.replace('\'','')
    BIO_anno = BIO_anno.replace(',','')
    BIO_anno = BIO_anno.strip('[')
    BIO_anno = BIO_anno.strip(']')
    out = str(test_update['id'][line]) + ',' + BIO_anno + ',' + str(test_update['class'][line])+'\n'
    file.write(str(out))

algo_output=pd.read_csv('output1.csv')
for i in range(0,len(algo_output["id"])):
    algo_output['id'][i] = int (algo_output['id'][i])  #string转换为int
    algo_output['class'][i] = int (algo_output['class'][i])
file.close()
    


C:\Users\cs-demo\AppData\Local\Temp/ipykernel_67392/986796269.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  algo_output['id'][i] = int (algo_output['id'][i])  #string转换为int
C:\Users\cs-demo\AppData\Local\Temp/ipykernel_67392/986796269.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  algo_output['class'][i] = int (algo_output['class'][i])
